# Library importing

In [ ]:
import pandas as pd
import numpy as np

# Data Loading

In [ ]:
df = pd.read_csv('/kaggle/input/bank-loan/Bankloan.txt')
df

In [ ]:
df.info()

# *EDA : Exploratory Data Analysis*

In [ ]:
# Data profiling library installation
!pip install ydata_profiling

In [ ]:
from ydata_profiling import ProfileReport
# Profile Report generation
profile = ProfileReport (df,title= 'Bank loan EDA', type_schema = { 'ed':'categorical','default':'categorical' } )
# Save the Report
profile.to_file ("dataset_profile_report.html")


In [ ]:
profile

# Needed Preparation Actions

The obtained results from EDA suggest that the following data cleaning and preparation steps must be undertaken:

   * The 'age' column contains values that are out of range and should be converted to null.
   * The distribution of the 'age' suggests that values have been rounded, so it might be better to categorize it.
   * In the 'ed' column, the values '4' and '5' should be merged, as the frequency of '5' is low.
   * The 'default' column includes inconsistent codes and should be corrected
   * The missing values in 'age', 'ed' and 'income' should be imputed.
   * The outliers should be detected and treated.
   * The 'employ','address','income', 'debtinc', 'creddebt' and 'otherdebt' have skewed distribution and should be transformed or categorized.
   * The target field is imbalanced; it should be taken into consideration during modeling

# Comparison EDA 

In [ ]:


from ydata_profiling import ProfileReport

df_default_0 = df[df.default == "0"]
df_default_1 = df[df.default == "1"]

# Generate a profile report
profile_0 = ProfileReport(df_default_0, title="Bankloan EDA 0",minimal=True,type_schema = {"Ed": "categorical", "Default": "categorical"})
profile_1 = ProfileReport(df_default_1, title="Bankloan EDA 1",minimal=True, type_schema = {"Ed": "categorical", "Default": "categorical"})

comparison_report = profile_0.compare(profile_1)
comparison_report.to_file("comparison.html")



In [ ]:
comparison_report

In [ ]:
df.info()

# Data Splitting

In [ ]:
target = df.iloc[:,-1]
inputs = df.iloc[:,0:-1]

# Categorizing

In [ ]:
columns = inputs.columns
# print(columns)
categorical_indices = [1]
categorical_fields = [columns[i] for i in categorical_indices]
continuous_fields = [j for j in columns if j not in categorical_fields]

# Feature Screening:

In [ ]:
# For Continuous fields:
#Define a Minimum value for coefficient of variation:
min_cv = 0.1
cv_values = inputs[continuous_fields].std() / inputs[continuous_fields].mean()
# print(cv_values)
selected_columns = cv_values[cv_values < 0.1].index
# print(selected_columns)
filtered_con = inputs[selected_columns]
inputs_con = inputs[continuous_fields].drop(selected_columns,axis=1)
print(inputs_con)

In [ ]:
threshold1 = 95

mode_category = (inputs[categorical_fields].apply(lambda x: x.dropna().value_counts().max() / len(x))) *100
print(mode_category)
selected_categorical_columns = mode_category[mode_category > threshold1].index
print('selected_categorical_columns=',selected_categorical_columns)
mode_filtered_inputs = inputs[selected_categorical_columns]
print('mode_filtered_inputs=',mode_filtered_inputs)
inputs_cat = inputs[categorical_fields].drop(mode_filtered_inputs,axis=1)
print(inputs_cat)

In [ ]:
threshold = 90
distinct_percentage = (inputs_cat.apply(lambda x: x.dropna().nunique() / x.count())*100) 
print('distinct_percentage=',distinct_percentage)
selected_categorical_columns = distinct_percentage[distinct_percentage > threshold ].index
print('selected_categorical_columns=',selected_categorical_columns)
distinct_filtered_inputs = inputs_cat[selected_categorical_columns]
print('distinct_filtered_inputs=',distinct_filtered_inputs)
inputs_cat = inputs_cat.drop(distinct_filtered_inputs, axis=1)
print(inputs_cat)

In [ ]:
filtered_df = pd.concat([inputs_con,inputs_cat,target],axis=1)
pd.set_option("display.width",500)
# filtered_df = pd.concat([target],axis=1)
print(filtered_df.describe())
print(filtered_df.columns)

# Logical Range Checking

In [ ]:
column_ranges = {'age':(18,70),'employ':(0,31),'address':(0,80),'income':(0,1000),'debtinc':(0,100),'creddebt':(0,30),'othdebt':(0,30)}

print(column_ranges.items())

for col, (min_val,max_val) in column_ranges.items():
    filtered_df[col] = filtered_df[col].apply(lambda x: x if min_val <= x <=max_val else None)

print(filtered_df)
print(filtered_df.describe())
print(filtered_df.info())

# Frequency Table Function

In [ ]:
import numpy as np

def frequency_table(variable):
    unique_elements, counts = np.unique(variable.dropna(),return_counts=True)
    percentage = (counts / (len(variable.dropna()))) * 100
    value_counts_and_percentage = zip(unique_elements,counts,percentage)
    
    for i, j, k in value_counts_and_percentage:
        print(f"{i:<5}: Counts:{j:>4}, Percentage:{k:>5.2f}%")
    return 

# Inconsistency Checking for Categorical fields:

In [ ]:
print('Frequency Table for Default:')
frequency_table(filtered_df['default'])

In [ ]:
filtered_df['default'] = filtered_df['default'].replace(["'0'",':0'],'0')
print('Frequency Table for Default:')
frequency_table(filtered_df['default'])

# Save

In [ ]:
filtered_df.to_csv("/kaggle/working/Bankloan_cleanedv1.csv")